In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd 

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util # type: ignore

util.startLoop()

# Get Base Data

In [21]:
# get portfolio, undsymbols and openorders
import numpy as np

from fast_prices import df_iv
from snp import get_snp_unds
from utils import (get_ib, get_open_orders,
                   how_many_days_old, pickle_me, quick_pf)

unds_path = ROOT/'data'/'snp_unds.pkl'

if how_many_days_old(unds_path) > 0.2:
    unds = get_snp_unds()
    pickle_me(unds, unds_path)
else:
    unds = pd.read_pickle(unds_path)

with get_ib('SNP') as ib:
    unds_iv=ib.run(df_iv(ib=ib, stocks=unds, msg='first run ivs'))

    no_price=unds_iv[unds_iv[['price', 'hv', 'iv']].isnull().any(axis=1)].symbol.to_list()
    second_unds_iv = ib.run(df_iv(ib=ib, stocks=no_price, sleep_time=10, msg='second run ivs'))

    pf = quick_pf(ib)
    oo = get_open_orders(ib)

# Set symbol as index for both dataframes
cols = ['symbol', 'price', 'iv', 'hv']

unds_iv = unds_iv[cols].set_index('symbol')
second_unds_iv = second_unds_iv[cols].set_index('symbol')

# Update unds_iv with non-null values from second_unds_iv 
unds_iv.update(second_unds_iv)

# unds_iv = unds_iv.set_index('symbol')[['hv', 'iv', 'price']]
unds_iv.columns = ['und_' + col for col in unds_iv.columns]
unds_iv = unds_iv.reset_index()

# ... add und_price
pf = pf.merge(unds_iv, on='symbol', how='left')


# ...temp store the pf, oo
pf_path = ROOT / 'data' / 'pf.pkl'
oo_path = ROOT / 'data' / 'oo.pkl'

pickle_me(pf, pf_path)
pickle_me(oo, oo_path)

second run ivs: 100%|██████████| 165/165 [00:46<00:00,  3.58chunk/s]


# Classify positions
Positions are classified as follows:
- `cwp`: the perfect position that is protected and has a cover.
- `exposed`: stocks that need to be covered and protected.
- `uncovered`: stocks that need to be only covered by options.
- `unprotected`: stocks that need to be only protected by options.
- `orphaned`: options that have no underlying stocks positions.
- `covering`: options that are covering positions.
- `protecting`: options that are portecting positions.

In [60]:
# Sort by covered with protection pairs
right_order = {'C': 0, '0': 1, 'P': 2}

pf = pf.sort_values(
    by=['symbol', 'right'],
    key=lambda x: x.map(right_order) if x.name == 'right' else x
)

# Initialize strategy field with blank underscore
pf['strategy'] = 'tbd'

# Filter for options only
opt_pf = pf[pf.secType == 'OPT']

# Group by symbol and expiry to find matching calls and puts
straddled = (opt_pf.groupby(['symbol', 'expiry', 'strike'])
                      .filter(lambda x: (
                          # Must have exactly 2 rows (call and put)
                          len(x) == 2 and 
                          # Must have both C and P
                          set(x['right']) == {'C', 'P'} and
                          # Position signs must match
                          np.sign(x['position'].iloc[0]) == np.sign(x['position'].iloc[1])
                      )))

# Update strategy field for straddles
pf.loc[pf.index.isin(straddled.index), 'strategy'] = 'straddled'

# Filter for stocks and their associated options
cwp = (pf.groupby('symbol')
                      .filter(lambda x: (
                          # Must have exactly one STK row
                          (x.secType == 'STK').sum() == 1 and
                          # Must have 1 or 2 OPT rows
                          (x.secType == 'OPT').sum() in [1, 2]
                      )))

# Update strategy field for covered calls/puts
pf.loc[pf.index.isin(cwp[cwp.right == 'C'].index), 'strategy'] = 'covering'
pf.loc[pf.index.isin(cwp[cwp.right == 'P'].index), 'strategy'] = 'protecting'

# Update strategy field for stocks with both covering and protecting
stocks_with_both = pf[(pf.secType == 'STK') & 
                      pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                      pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_with_both.index, 'strategy'] = 'cwp'

# Update strategy field for stocks with covering but no protecting
stocks_covered_only = pf[(pf.secType == 'STK') &
                        pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                        ~pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_covered_only.index, 'strategy'] = 'unprotected'

# Update strategy field for stocks with protecting but no covering  
stocks_protected_only = pf[(pf.secType == 'STK') &
                          ~pf.symbol.isin(pf[(pf.strategy == 'covering')].symbol) &
                          pf.symbol.isin(pf[(pf.strategy == 'protecting')].symbol)]
pf.loc[stocks_protected_only.index, 'strategy'] = 'uncovered'


# Update strategy field for orphaned options
pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'OPT'), 'strategy'] = 'orphaned'

# Update strategy field for exposed stock positions
pf.loc[(pf.strategy == 'tbd') & (pf.secType == 'STK'), 'strategy'] = 'exposed'


In [61]:
# Check for null values in price, und_hv, and und_iv columns
null_rows = pf[pf[['und_price', 'und_hv', 'und_iv']].isnull().any(axis=1)]
print("Rows with null values in price, historical vol or implied vol:")
display(null_rows)


Rows with null values in price, historical vol or implied vol:


,secType,conId,symbol,expiry,strike,right,contract,position,mktPrice,mktVal,avgCost,unPnL,rePnL,und_price,und_iv,und_hv,strategy
49,OPT,729924008,IEFA,20241115,75.0,C,"Option(conId=729924008, symbol='IEFA', lastTra...",-4.0,0.20,-78.42,70.48,203.51,0.0,NaN,NaN,NaN,covering
48,STK,115826907,IEFA,,0.0,0,"Stock(conId=115826907, symbol='IEFA', right='0...",400.0,73.77,29508.00,66.12,3059.72,0.0,NaN,NaN,NaN,unprotected


# Cook orders for existing positions
- Exposed and Unprotected stocks should be protected
- Exposed and Uncovered stocks should be covered
- Orphaned stocks should be liquidated

## For rest of the symbols
- Symbols with announcements in a week need to be straddled
- Remaining ones should have naked puts